
<div style="background-color:white; text-align:center; padding:20px;">
    <h2 style="color:black; font-family: Verdana, sans-serif;"><strong>Warlord</strong></h2>
    <p style="font-size: 14px; color: black; font-family: Verdana, sans-serif;"> 
        <table style="margin: auto; border-collapse: collapse;">
            <tr>
                <th style="border: 0;">Names</th>
                <th style="border: 0;">GitHub Username</th>
            </tr>
            <tr>
                <td style="border: 0;">Vince Ammerlaan</td>
                <td style="border: 0;">Vince16270</td>
            </tr>
            <tr>
                <td style="border: 0;">Tommi Lander</td>
                <td style="border: 0;">tommi7</td>
            </tr>
            <tr>
                <td style="border: 0;">Nima Ghafar</td>
                <td style="border: 0;">NimaGhafar</td>
            </tr>
            <tr>
                <td style="border: 0;">Isa Dijkstra</td>
                <td style="border: 0;">IsaD01</td>
            </tr>
        </table>
    </p>
</div>

<div style="display: flex; justify-content: center; align-items: center; margin-top: 10px;">
    <img src="photo/image1.png" alt=" photo needs to be added" style="width: 1000px; height: auto;">
</div>


- Docent: Vikram Radhakrishnan
- Datum: 20-06-2025

---
<div style="background-color:white; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Inhoudsopgave </strong></h2>
</div>
<ul style="padding: 0; list-style: none;">
    <li style="text-align: center;"><a href="#1.0">Chapter 1: Het project</a></li>
    <li style="text-align: center;"><a href="#2.0">Chapter 2: Packages en functies inladen</a></li>
    <li style="text-align: center;"><a href="#3.0">Chapter 3: Trainen van de agents</a></li>
    <li style="text-align: center;"><a href="#4.0">Chapter 4: Resultaten van het trainen</a></li>
    <li style="text-align: center;"><a href="#5.0">Chapter 5: Conclusie</a></li>
    <li style="text-align: center;"><a href="#5.0">Chapter 6: Bronnenlijst</a></li>
</ul>

---
<div style="background-color:white; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Chapter 1: Het project </strong></h2>

</ul>

In deze opdracht richten we ons op het computerspel Warlords. In dit spel beschermd elke speler een kasteel dat zich in een van de vier hoeken van het scherm bevindt. Vanuit het midden van het scherm komt een bal die wordt teruggekaatst om de kastelen van de tegenstanders te vernietigen. Elke speler heeft een schild waarmee hij de bal kan afweren of richten op vijanden.

Ons doel is om meerdere slimme agents te ontwikkelen die elk zelfstandig leren spelen via multi-agent reinforcement learning. Dat betekent dat elke agent leert op basis van zijn eigen ervaringen, maar ook rekening moet houden met het gedrag van andere agents. De omgeving is dynamisch en strategisch: je moet niet alleen goed verdedigen, maar ook slim aanvallen.

Warlords is bij uitstek geschikt voor dit soort leerstrategieën, omdat het meerdere spelers bevat die tegelijk handelen en op elkaar reageren. Dat maakt het leerproces complex en interessant: agents moeten niet alleen leren van hun eigen fouten, maar ook anticiperen op anderen.

Het spel eindigt op twee manieren:

- Je wint door als laatste overgebleven speler in het spel te zijn.
- Je verliest als je kasteel volledig vernietigd is door de bal.

**Wat is het probleem?**

We willen een slimme agent ontwikkelen die zelfstandig leert om het spel Warlords goed te spelen. Deze agent moet:

- Leren hoe de bal beweegt en hoe hij daarop moet reageren
- Ontwikkelen hoe je effectief verdedigt én aanvalt
- Tactisch aanvallen uitvoeren om de vijandelijke kastelen te verzwakken
- Opletten op meerdere tegenstanders tegelijk en zich aanpassen aan hun gedrag

**Wat is het doel?**

De agent moet:

- Zelf leren van hun ervaringen en hun gedrag continu aanpassen
- Beter presteren dan agents die willekeurig bewegen of vaste strategieën gebruiken
- Verschillende strategieën ontwikkelen afhankelijk van tegenstanders
- Leren overleven in een complexe omgeving en uiteindelijk winnen


---
<div style="background-color:white; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Chapter 2: Packages en functies inladen </strong></h2>

</ul>

In [1]:
import yaml, time, numpy as np, torch, pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from src.env.warlords_env import make_env
from src.agents.agent import PPOAgent, RandomAgent
from src.utils import device         

In [2]:
print("PyTorch:", torch.__version__, "| device:", device)

PyTorch: 2.6.0 | device: mps


In [3]:
# Config laden 
cfg_path = "configs/ppo_warlords.yaml"
with open(cfg_path) as f:
    cfg = yaml.safe_load(f)

cfg

{'seed': 42,
 'ppo': {'max_timesteps': 3000000,
  'gamma': 0.99,
  'lr': 0.00025,
  'eps_clip': 0.1,
  'K_epochs': 4,
  'save_path': 'models/ppo_warlords.pt',
  'log_interval': 10000}}

---
<div style="background-color:white; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Chapter 3: Trainen van de agents </strong></h2>

</ul>

**PPO Agent: Proximal Policy Optimization**

Wij hebben ervoor gekozen om een PPO-agent te trainen. PPO staat voor Proximal Policy Optimization en is een state-of-the-art algoritme dat agents in staat stelt om zelfstandig te leren door middel van trial-and-error in een omgeving.

De drie belangrijkste voordelen van PPO zijn:

- Eenvoud
- Stabiliteit
- Steekproefefficiëntie


Het PPO-algoritme werkt volgens de volgende principes:

- *Policy Gradient Methods*:
PPO leert direct welk gedrag (actie) optimaal is in een bepaalde situatie (state), in plaats van alleen in te schatten hoe goed een situatie op zichzelf is.
- *Objective Function*:
Het algoritme probeert het gedrag van de agent zó aan te passen dat deze op de lange termijn zoveel mogelijk beloning ontvangt uit de omgeving.

- *Clipped Surrogate Objective*:
PPO voorkomt dat de agent na één leerstap drastisch ander gedrag gaat vertonen. Grote aanpassingen in het gedrag worden ‘geclipt’, waardoor het leerproces stabiel blijft.

- *Multiple Epochs & Mini-Batch Updates*:
De agent verzamelt ervaringen (observaties, acties, beloningen) in de omgeving. Deze gegevens worden vervolgens herhaaldelijk gebruikt tijdens het leerproces, opgesplitst in kleine, efficiënte mini-batches.

- *Value Function Estimation*:
PPO maakt ook gebruik van een waarde-inschatting van situaties (de ‘value function’), wat helpt om het leerproces minder schommelend en efficiënter te maken.

- *Parallelization*:
Het algoritme is geschikt voor parallel leren (bijvoorbeeld op meerdere CPU’s of GPU’s), wat de trainingssnelheid aanzienlijk verhoogt.

(DhanushKumar, 2024)

In [6]:
import ale_py, textwrap
games = [g for g in ale_py.list_games() if "warlord" in g]
print(textwrap.fill(" ".join(games), 80))
# → warlord

AttributeError: module 'ale_py' has no attribute 'list_games'

In [4]:
from src.env.warlords_env import make_env
env = make_env(42)
print("OK - action space:", env.action_space)
env.close()

NameNotFound: Environment `Warlord` doesn't exist in namespace ALE.

In [ ]:
env = make_env(cfg["seed"])

obs_dim    = env.observation_space.shape[0]
action_dim = env.action_space.n
agent      = PPOAgent(obs_dim, action_dim, cfg["ppo"])

# Logstructuur
log_dir = Path("../logs")
log_dir.mkdir(exist_ok=True)
reward_log = []
loss_log   = []           

# --- Train loop -----------------------------------------
episodes, timestep = 0, 0
obs = env.reset()
pbar = tqdm(total=cfg["ppo"]["max_timesteps"], desc="Timesteps")

while timestep < cfg["ppo"]["max_timesteps"]:
    action = agent.select_action(obs)
    obs, r, done, _ = env.step(action)

    agent.store_reward(r, done)
    timestep += 1
    pbar.update(1)

    if done:
        episodes += 1
        reward_log.append({"episode": episodes, "reward": sum(agent.buffer["rewards"])})
        agent.update()                 # ← hier gebeurt PPO-update
        obs = env.reset()

# Save model & logs
model_path = Path(cfg["ppo"]["save_path"])
model_path.parent.mkdir(exist_ok=True)
agent.save(model_path)

pd.DataFrame(reward_log).to_csv(log_dir / "rewards.csv", index=False)
print(f"Training klaar – {episodes} episodes, model: {model_path}")

---
<div style="background-color:white; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Chapter 4: Resultaten van het trainen </strong></h2>

</ul>

In [ ]:
# --- Laad logs ------------------------------------------
log = pd.read_csv(log_dir / "rewards.csv")
log["smooth"] = log.reward.rolling(50).mean()

# --- Plot ------------------------------------------------
fig, ax = plt.subplots(figsize=(8,4))
ax.plot(log.episode, log.reward, alpha=.3, label="Episode reward")
ax.plot(log.episode, log.smooth, lw=2, label="Rolling mean (50)")
ax.set_xlabel("Episode"); ax.set_ylabel("Reward")
ax.set_title("PPO on Warlords – training curve")
ax.legend(); plt.show()

In [ ]:
# --- Histogram eindscore --------------------------------
log.reward.plot.hist(bins=30)
plt.title("Verdeling episode-reward")
plt.xlabel("Reward"); plt.show()

In [ ]:
from moviepy.editor import ImageSequenceClip

env = make_env(cfg["seed"])
agent.load(model_path)          # geladen op correct device

frames = []
obs = env.reset()
done = False
while not done:
    act = agent.select_action(obs)
    obs, _, done, _ = env.step(act)
    frame = env.render(mode="rgb_array")
    frames.append(frame)

gif_path = log_dir / "ppo_warlords.gif"
ImageSequenceClip(frames, fps=15).write_gif(gif_path)
print("GIF opgeslagen:", gif_path)

---
<div style="background-color:white; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Chapter 5: Conclusie </strong></h2>
</div>

---
<div style="background-color:white; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Chapter 6: Bronnenlijst </strong></h2>
</div>

- DhanushKumar. (2024, 12 mei). PPO Algorithm - DhanushKumar - Medium. Medium. https://medium.com/@danushidk507/ppo-algorithm-3b33195de14a

Mogelijke bronnen:

- https://arxiv.org/pdf/1707.06347
- https://lilianweng.github.io/posts/2018-04-08-policy-gradient/
- https://spinningup.openai.com/en/latest/algorithms/ppo.html

